# Load Data

In [10]:
import util
from importlib import reload
import dataset as d
import model as m
import loss as l
reload(util)
reload(l)
reload(d)
reload(m)

de = util.DataExtractor()
gt_df = de.croppedData()

ds = d.CroppedDummyset(gt_df)
myDataset = d.CroppedDataset(gt_df)


Saving to: c:\Users\alexh\Desktop\cv2\obb_anns_hausarbeit\ds2_dense\ds2_dense\gt_space.json


## Test unit_precision

In [27]:
import dataset

ds = dataset.CroppedDummyset(gt_df)

choices = ["one","half","inverse_half","vertical_half","quarter","checker","random25"]
def sweep(ds):
    res = {}
    for p in choices:
        P = ds.dummyTensor(p)
        row = {}
        for t in choices:
            T = ds.dummyTensor(t)
            iou = eval.pred_to_iou(P, T)
            row[t] = eval.unit_precision(P, T, iou)
        res[p] = row
    return res

mat = sweep(ds)
for p,row in mat.items():
    print(p, ["%s:%.3f"%(t,v) for t,v in row.items()])

one ['one:1.000', 'half:0.500', 'inverse_half:0.500', 'vertical_half:0.500', 'quarter:0.250', 'checker:0.500', 'random25:0.256']
half ['one:1.000', 'half:1.000', 'inverse_half:0.000', 'vertical_half:0.500', 'quarter:0.500', 'checker:0.500', 'random25:0.260']
inverse_half ['one:1.000', 'half:0.000', 'inverse_half:1.000', 'vertical_half:0.500', 'quarter:0.000', 'checker:0.500', 'random25:0.252']
vertical_half ['one:1.000', 'half:0.500', 'inverse_half:0.500', 'vertical_half:1.000', 'quarter:0.500', 'checker:0.500', 'random25:0.275']
quarter ['one:1.000', 'half:1.000', 'inverse_half:0.000', 'vertical_half:1.000', 'quarter:1.000', 'checker:0.500', 'random25:0.285']
checker ['one:1.000', 'half:0.500', 'inverse_half:0.500', 'vertical_half:0.500', 'quarter:0.250', 'checker:1.000', 'random25:0.271']
random25 ['one:1.000', 'half:0.507', 'inverse_half:0.493', 'vertical_half:0.537', 'quarter:0.278', 'checker:0.529', 'random25:1.000']


# Debug Precision Tests

In [5]:
import sys, importlib

# 1) Verify which file you're importing
import eval
print("FILE:", eval.__file__)

# 2) Hard-reload
sys.modules.pop('eval', None)
eval = importlib.import_module('eval')

# 3) Make sure runTests calls debug
eval.debug_pred_to_iou()   # should print stuff
eval.runTests(gt_df)

FILE: c:\Users\alexh\Desktop\cv2\obb_anns_hausarbeit\eval.py
max |bp-bt|: 0.0
IoU diag mean/min/max: 1.0 1.0 1.0
oh, ho, oo = 0.5 1.0 1.0
conf≥thr: 3200 pc==tc: 3200 tp_m: 3200 iou>0: 3200
conf≥thr: 1600 pc==tc: 3200 tp_m: 1600 iou>0: 3200


# Random Model Output

In [29]:
import torch
import torchvision.models as models
import model as m
import dataset as d
import eval
import config as c
from importlib import reload
reload(eval)
reload(d)
reload(m)

# Assuming gt_df is defined from previous context
ds = d.CroppedDataset(gt_df)  # Use the real dataset

ds2 = d.CroppedDummyset(gt_df)
one = ds2.dummyTensor("one")
half = ds2.dummyTensor("half")

class FixedYOLOResNet(m.YOLOResNet):
    def __init__(self):
        super().__init__()
        backbone = models.resnet18(weights=None)
        # Adapt for grayscale input
        backbone.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=3, padding=3, bias=False)

        # Set strides to 1 only for layer2 and layer3 to achieve total downsampling factor of 12 (3*2*2=12)
        for layer in [backbone.layer2, backbone.layer3]:
            for block in layer:
                if hasattr(block, 'conv1'):
                    block.conv1.stride = (1, 1)
                if hasattr(block, 'downsample') and block.downsample is not None:
                    block.downsample[0].stride = (1, 1)

        # Backbone up to the feature map (layer4 keeps default stride=2)
        self.backbone = torch.nn.Sequential(
            backbone.conv1,
            backbone.bn1,
            backbone.relu,
            backbone.maxpool,
            backbone.layer1,
            backbone.layer2,
            backbone.layer3,
            backbone.layer4
        )

        # YOLO detection head
        num_features = 512
        self.head = torch.nn.Conv2d(num_features, self.A * (5 + self.C), kernel_size=1, stride=1, padding=0)

m1 = m.YOLOResNet()#FixedYOLOResNet()
m1.eval()  # Ensure evaluation mode

precisions = []
for i in range(10):
    image, target = ds[i]
    # Image is [H, W], add channel
    image = image.unsqueeze(0)  # [1, H, W]
    with torch.no_grad():
        pred = m1(image.unsqueeze(0))  # [1, 1, H, W] -> model -> [1, N, N, A, 5+C]
        pred = pred.squeeze(0)  # [N, N, A, 5+C]
    iou = eval.pred_to_iou(pred, target)
    prec = eval.unit_precision(pred, target, iou)
    # iou = eval.pred_to_iou(one, half)
    # prec = eval.unit_precision(one, half, iou)
    precisions.append(prec)
    print(f"Sample {i}: Precision = {prec:.3f}")

average_precision = sum(precisions) / len(precisions) if precisions else 0
print(f"Average precision over 10 samples: {average_precision:.3f}")

Sample 0: Precision = 0.000
Sample 1: Precision = 0.000
Sample 2: Precision = 0.000
Sample 3: Precision = 0.000
Sample 4: Precision = 0.000
Sample 5: Precision = 0.000
Sample 6: Precision = 0.000
Sample 7: Precision = 0.000
Sample 8: Precision = 0.000
Sample 9: Precision = 0.000
Average precision over 10 samples: 0.000
